# Lab 3: Creating Branches and Applying Cuts

## Table of Contents
1. **[Creating Branches](#creating-branches)**  
    1.1. [Transforming Momentum Coordinate Systems](#trans-momentum)  
    1.2. [Calculating ($P_x, P_y, P_z$)](#calc-momentum)  
    1.3. [Exercise](#exercise-1.3)
2. **[Cuts on TTree Branches](#cuts)**  
    2.1. [Applying Cuts](#apply-cuts)  
    2.2. [Exercise](#exercise-2.2)

## 1. Creating Branches <a name="creating-branches" />
Often, you find you will want to create a new distribution from existing distributions in a ROOT file to calculate certain physical values that aren't already calculated. This can involve transforming the coordinate system of the momentum, calculating invariant masses of particles, or writing the classification of an event from the decision of a neural network.

### 1.1 Transforming Momentum Coordinate System <a name="trans-momentum" />
The detector does not store the momentum of each particle as $(P_x, P_y, P_z)$. Instead, momentum information is stored as $(P_t, \phi, \eta)$. These coordinate systems are related by the following equations:
$$P_x = P_t \times cos(\phi)$$
$$P_y = P_t \times sin(\phi)$$
$$P_z = P_t \times sinh(\eta)$$
In order to transform our momentum from one coordinate system to another, we will be using root_pandas to work with the events as arrays. We will import the ROOT file to a panda dataframe, then convert the dataframe to a numpy array. This allows us to perform calculations extremely quickly. If left as dataframes, this conversion takes around 20 minutes with our dataset. In numpy arrays, it takes about a second.

In [1]:
!pip install root_pandas -q

In [27]:
import ROOT
import pandas as pd
import numpy as np
from root_pandas import read_root, to_root

In [23]:
#Import ROOT file into TTree
pandas_dataframe = read_root("../../Datasets/TTbar/ee_ttbarsignalplustau.root", key = "ttBar_treeVariables_step8")
print("dataframe imported!")

dataframe imported!


Now we have a dataframe with every branch (or column in panda's lingo) from the `TFile`'s `TTree`. Before we start calculating $P_x, P_y,$ and $P_z$, we must add new columns to our dataframe for the values to be stored in. You can define this list by hand, but it can be tedious. Here is a general approach to creating this list of names from a predefined list of particles.

In [24]:
#List of input particles
particles = ["top", "tbar", "l", "lbar", "b", "bbar"]

output_column_names = []
for particle in particles:
    output_column_names.append("{}_px".format(particle))
    output_column_names.append("{}_py".format(particle))
    output_column_names.append("{}_pz".format(particle))
print("Output column names: ", output_column_names)

('Output column names: ', ['top_px', 'top_py', 'top_pz', 'tbar_px', 'tbar_py', 'tbar_pz', 'l_px', 'l_py', 'l_pz', 'lbar_px', 'lbar_py', 'lbar_pz', 'b_px', 'b_py', 'b_pz', 'bbar_px', 'bbar_py', 'bbar_pz'])


The function ``format()`` is a method for strings where it changes ``{}`` in the string to the argument passed to it. This makes it quick and easy to generate our output branch names, alleviating the need to type out every single one. We can simply make a list of every particle we want to work with, and the script will create the output column names for us.  

Now we need $(P_t, \phi, \eta)$ for each particle. Our current dataframe has all the original distributions from the `TTree`. Let's get these distributions specifically from the original dataframe and create a new dataframe with just these distributions. 

In [25]:
##Rapidity and eta are the same thing, but each particle is labeled differently in the TTree
#For top and tbar
variables_top = ["{}_pt", "{}_phi", "{}_rapidity"]
#for l, lbar, b, bar
variables = ["{}_pt", "{}_phi", "{}_eta"]

list_dists = []
for particle in particles:
    #Check if particle is a top or not
    if particle == "top" or particle == "tbar": 
        for variable in variables_top:
            list_dists.append(pd.DataFrame(pandas_dataframe.get(variable.format(particle))))
    else:
        for variable in variables:
            list_dists.append(pd.DataFrame(pandas_dataframe.get(variable.format(particle))))

#Combine DataFrames for all particles
input_dataframe = pd.concat(list_dists, axis = 1)

To create a new dataframe with only $P_t, \eta,$ and $\phi$, we first need to grab each column into a single dataframe. This is accomplished by using ``dataframe.get("column name")``. However, this returns a numpy array, so we create a dataframe from the array. Then to create a new dataframe with all the desired distributions we use ``pd.concat()``, which takes a list of dataframes and returns a single dataframe composed of the lists of dataframes. The option ``axis = 1`` ensures they are concatonated along the correct axis.  

### 1.2 Calculating ($P_x, P_y, P_z$) <a name="calc-momentum" />
Now that we have the distributions, we can perform calculations. We will work with each particle individually and write the output $P_x, P_y,$ and $P_z$ to an output frame. We will later concatonate this output frame with out original dataframe. Once concatonated, we can use ``to_root()`` to write our dataframe to a ROOT file.

In [32]:
#Initialize output frame
output_dataframe = pd.DataFrame(np.zeros((len(pandas_dataframe),1)), columns = ['drop'])

for particle in particles:
    #Top particles' eta labelled differently in the TTree
    if particle == "top" or particle == "tbar":
        part_pt_array = pandas_dataframe["{}_pt".format(particle)]
        part_phi_array = pandas_dataframe["{}_phi".format(particle)]
        part_eta_array = pandas_dataframe["{}_rapidity".format(particle)]
    else:
        part_pt_array = pandas_dataframe["{}_pt".format(particle)]
        part_phi_array = pandas_dataframe["{}_phi".format(particle)]
        part_eta_array = pandas_dataframe["{}_eta".format(particle)]

    #Output column names
    part_px_str = "{}_px".format(particle)
    part_py_str = "{}_py".format(particle)
    part_pz_str = "{}_pz".format(particle)

    #Compute px, py, and pz and create pandas dataframe from numpy array
    part_px_array = np.multiply(part_pt_array,np.cos(part_phi_array))
    part_px_array.name = part_px_str
    part_py_array = np.multiply(part_pt_array,np.sin(part_phi_array))
    part_py_array.name = part_py_str
    part_pz_array = np.multiply(part_pt_array,np.sinh(part_eta_array))
    part_pz_array.name = part_pz_str
    
    part_px_df = pd.DataFrame(part_px_array, columns = [part_px_str])
    part_py_df = pd.DataFrame(part_py_array, columns = [part_py_str])
    part_pz_df = pd.DataFrame(part_pz_array, columns = [part_pz_str])
    part_3vec_dataframe = pd.concat([part_px_array, part_py_array, part_pz_array], axis = 1)

    #Concatonate particle px, py, and pz to output dataframe
    output_dataframe = pd.concat([output_dataframe, part_3vec_dataframe], axis = 1)


Now ``output_dataframe`` has the $P_x, P_y,$ and $P_z$ of each particle we asked the script to calculate. Before we concatonate this frame with our original frame, we must drop the "drop" column we used to initialize the frame. Once this is done, we can concatonate the two frames and write the final frame to a ROOT file. 

In [31]:
output_frame = output_frame.drop(["drop"], axis = 1)
output_frame = pd.concat([pandas_dataframe, output_frame], axis = 1)
to_root(output_frame, "../../Datasets/TTbar/ee_ttbarsignalplustau_3vec.root", key = 'DecayTree')

### 1.3 Exercise <a name="exercise-1.3" />
With the ROOT file created above, calculate the magnitude of the momentum for each particle and create a new ROOT file with your new and original distributions. Use ``np.square()`` to square the array's element-wise, and ``np.sqrt()`` to take the square root element-wise.

In [34]:
#Import input dataframe

#List of input particles
particles = ["top", "tbar", "l", "lbar", "b", "bbar"]

#List of input variables
variables = ["{}_px", "{}_py", "{}_pz"]

#Initialize output dataframe


for particle in particles:
    #Create numpy array from dataframe
    
    #Create output branch (column) names using .format()
    
    #Compute p_mag
    
    #Concatonate p_mag to output dataframe
    
    
#Create new ROOT file
    

IndentationError: expected an indented block (<ipython-input-34-7c8c6732ec31>, line 22)

## 2. Cuts on TTree Branches <a name="cuts" />

During an analysis, one of your objectives is to remove background from your data while retaining signal. Removing events from your data is called applying "cuts". In this lab, we will not discuss how to decide what is a good cut. We will simply go over how to apply them and create an output ROOT file.

### 2.1 Applying Cuts <a name="apply-cuts" />
In ROOT, cuts are given as strings. You define the branch you wish to cut on, give a boolean operator, and the value you wish to cut at. You can use ``&&`` to pass multiple cuts at once. An example of a cut string is ``"top_pt > 0 && tbar_pt > 0"``. You can apply no cut by defining ``selection = "1"``. Let's apply the first cut to our ROOT file.

In [1]:
from ROOT import TFile, TTree, TObject

#Define what cut to apply to the data
selection = "top_pt >= 100 && tbar_pt >= 100"

#Retreieve ROOT file and TTree
source_root = TFile.Open("../../Datasets/TTbar/ee_ttbarsignalplustau.root")
source_tree = source_root.Get("ttBar_treeVariables_step8")

#Create Destination ROOT file for TTree
new_root = TFile("../../Datasets/TTbar/ee_ttbarsignalplustau_ttbar_pt_cut.root", "RECREATE")

#Cut and Copy Tree
cut_tree = source_tree.CopyTree(selection)

#Write cut Tree to a ROOT file
cut_tree.Write("DecayTree", TObject.kWriteDelete)

#Write ROOT file
new_root.Write()
new_root.Close()

Welcome to JupyROOT 6.17/01


``TTree.CopyTree(selection)`` is a `TTree` method which returns a copy of a `TTree` with the cut defined in selection applied. It is important that you create the output root file, before you call ``TTree.CopyTree()``. When you call ``TTree.CopyTree()``, ROOT internally assigns the returned `TTree` to a ROOT file. If you call ``TTree.CopyTree()`` before you create the output ROOT file, the `TTree` will not be assigned to the output ROOT file and cannot be written.

After a cut you will have less events than you started with. Thus we define the efficiency of a cut as: $$efficiency = \frac{N_{after}}{N_{before}}$$
where $N$ is the number of events. During an analysis you will want to keep track of the effeciency of every cut you apply. You could go into `TBrowser` and look at the number of entries at every stage, but is easier to have your script give you the efficiency. ``TTree.GetEntries()`` will give you the total number of events in a tree. We can calculate the efficiency quickly in our script:

In [ ]:
entries_before = float(source_tree.GetEntries())
entries_after = float(cut_tree.GetEntries())
efficiency = entries_after / entries_before
print("The efficiency of ", selection, "is: ", efficiency)

### 2.2 Exercise <a name="exercise-2.2" />
Starting from the original ROOT file ``ee_ttbarsignalplustau.root``, apply the following cuts:
$$
\begin{align*}
t_{pt} &> 0\\
t_{pt} &< 800\\
\bar{t}_{pt} &> 0\\
\bar{t}_{pt} &< 800\\
\end{align*}
$$

Then find the efficiency of each cut at each stage of selection, and the total combined efficiency of all the cuts.  

Hint: ``TTree.GetEntries()`` can also take a cut as an argument and give you the number of events which pass the cut. This bypasses having 4 ROOT files with different cut levels.